In [ ]:
from bioexplorer import BioExplorer, Protein, MovieMaker, Volume, Vector2, Vector3
url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)

In [ ]:
simulation_guid = 2

resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
metabolites_folder = pdb_folder + 'metabolites/'

representation=be.REPRESENTATION_ATOMS_AND_STICKS

scene_size = Vector3(200.0, 500.0, 200.0)
membrane_size = Vector2(scene_size.x, scene_size.z)
membrane_height = scene_size.y / 50.0


## Metabolites

In [ ]:
import psycopg2
import pandas as pd

db_host = 'localhost'
db_name = 'bioexplorer'
db_user = 'brayns'
db_password ='brayns'
db_schema = 'metabolism'

if False:
    db_host = 'bbpdbsrv01.bbp.epfl.ch'
    db_name = 'bioexplorer'
    db_user = 'bioexplorer'
    db_password = 'verole@2020'
    db_schema = 'metabolism'


db_connection_string = 'host=' + db_host + ' port=5432 dbname=' + db_name + \
              ' user=' + db_user + ' password=' + db_password

db_connection=psycopg2.connect(db_connection_string)
print('Connected to ' + db_name + '.' + db_schema)

In [ ]:
# Variables
def get_simulations():
    simulations = dict()
    sql_command = "SELECT guid, description FROM %s.simulations ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        simulations[str(data['guid'][i])] = data['description'][i]
    return simulations

def get_variables():
    variables = dict()
    sql_command = "SELECT guid, pdb_guid, description FROM %s.variable ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        variables[data['guid'][i]] = [data['pdb_guid'][i], data['description'][i]]
    return variables

def get_locations():
    locations = dict()
    sql_command = "SELECT guid, description FROM %s.location ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        locations[data['guid'][i]] = data['description'][i]
    return locations

def get_concentration(variable_guid, simulation_guid, frame, location_guid):
    sql_command = 'SELECT v.guid AS guid, c.concentration AS concentration FROM %s.variable as v, %s.concentration AS c WHERE c.variable_guid=%d AND v.guid=c.variable_guid AND c.timestamp=%d AND c.simulation_guid=%d AND v.location_guid=%d ORDER BY v.guid' % (db_schema, db_schema, variable_guid, frame, simulation_guid, location_guid)
    data = pd.read_sql(sql_command, db_connection)
    if(len(data)>0):
        return(float(data['concentration']))
    return 0.0    

In [ ]:
''' Dimensions are in nanometers '''

import math

location_areas = dict()
location_areas[0] = [0.45, Vector2(-20, -100)] # Cytosol of the neuron
location_areas[1] = [0.25, Vector2(20, 100)] # Cytosol of the astrocyte
location_areas[2] = [0.20, Vector2(-20, 20)] # Extracellular space
location_areas[3] = [0.0055, Vector2(0, 0)] # Capillaries (Not used)
location_areas[4] = [0.0, Vector2(0, 0)] # Synaptic part of the extracellular space (Not used)
location_areas[5] = [0.0575 * 0.45, Vector2(-100, -scene_size.x)] # Mitochondria of the neuron
location_areas[6] = [0.0575 * 0.25, Vector2(100, scene_size.x)] # Mitochondria of the astrocyte
location_areas[7] = [0.0, Vector2(0, 0)] # Vasculature (Not used)

fullNGVUnitVolumeInLiters = 2e-11 / 0.45
nanometersCubicToLiters = 1e-24
fullSceneVolumeInLiters = scene_size.x * scene_size.y * scene_size.z * nanometersCubicToLiters
ratio = fullSceneVolumeInLiters / fullNGVUnitVolumeInLiters

In [ ]:
import glob
import random
import os

def add_metabolites(pdb_guid, location_guid, frame):
    Avogadro = 6.02e23

    random_seed = 1

    file_name = metabolites_folder + str(pdb_guid) + '.pdb'
    concentration = 1e-3 * get_concentration(variable_guid, simulation_guid, frame, location_guid)
    nb_molecules = int(ratio * Avogadro * concentration * fullNGVUnitVolumeInLiters * location_areas[location_guid][0])
    if nb_molecules > 0:
        print('%d: %s: %d: %d' % (location_guid, variable_description, nb_molecules, nb_molecules))
        try:
            metabolite = Protein(
                sources=[file_name], load_bonds=True, load_hydrogen=True,
                load_non_polymer_chemicals=True, 
                occurences=nb_molecules)

            location_area = location_areas[location_guid][1]
            area_size = Vector3(scene_size.x, location_area.y - location_area.x , scene_size.z)
            area_position = Vector3(0.0, (location_area.y + location_area.x) / 2.0 , 0.0)

            volume = Volume(
                name=variable_description, size=area_size,
                protein=metabolite,
                random_position_seed=random_seed + frame, random_position_stength=area_size.y / 100.0,
                random_rotation_seed=random_seed + frame, random_rotation_stength=0.3
            )
            return be.add_volume(
                volume=volume, representation=representation,
                position=area_position, random_seed=random_seed)
        except Exception as e:
            print(e)


In [ ]:
def setup_frame(pdb_guid, location_guid, frame):
    be.reset()
    be.set_general_settings(model_visibility_on_creation=False)
    add_metabolites(pdb_guid, location_guid, frame)
    be.set_models_visibility(True)

In [ ]:
be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[-7.376115322113037, -69.28611755371094, 101.59373474121094],
    target=[-7.376115322113037, -69.28611755371094, 100.59373474121094])

In [ ]:
from IPython.display import clear_output

locations = get_locations()
variables = get_variables()

variable_guid = 9
location_guid = 0

variable = variables[variable_guid]

pdb_guid = variable[0]
variable_description = variable[1]
frame = 0
while True:
    clear_output()
    print(variable)
    setup_frame(pdb_guid, location_guid, frame % 300)
    mm.create_snapshot(
        renderer='bio_explorer'
        base_name = '%05d' % frame,
        path='/tmp',
        size=[960, 540], samples_per_pixel=8)
    frame += 1